1. Implement Shellsort which reverts to insertion sort. (Use the increment
sequence 7, 3, 1). Create a table (or preferably, a plot) for the total
number of comparisons made in the sorting the data for both cases (insertion
sort phase and shell sort phase). Explain why Shellshort is more effective than
Insertion sort in this case.

In [29]:
# https://www.geeksforgeeks.org/insertion-sort/
# Input is any length array
# After function call array is sorted
# Returns number of comparisons
def insertion_sort(input_array):
    comparisons = 0
    for i in range(1, len(input_array)):
        key = input_array[i]
        j = i-1
        while j >= 0 and key < input_array[j]:
            comparisons += 1
            input_array[j+1] = input_array[j]
            j -= 1
        comparisons += 1
        input_array[j+1] = key
    return comparisons
    

# https://www.geeksforgeeks.org/python-program-for-shellsort/
# Input is any length array
# After function call array is 7-sorted
# Returns number of comparisons
def shell7(input_array):
    comparisons = 0
    for i in range(7, len(input_array)):
        key = input_array[i]
        j = i
        while j >= 7 and input_array[j-7] > key:
            comparisons += 1
            input_array[j] = input_array[j-7]
            j -= 7
        comparisons += 1
        input_array[j] = key 
    return comparisons

# Input is any length array
# After function call array is 3-sorted
# Returns number of comparisons
def shell3(input_array):
    comparisons = 0
    for i in range(3, len(input_array)):
        key = input_array[i]
        j = i
        while j >= 3 and input_array[j-3] > key:
            comparisons += 1
            input_array[j] = input_array[j-3]
            j -= 3
        comparisons += 1
        input_array[j] = key 
    return comparisons

# Input is any length array
# After function call array is sorted
# Returns number of comparisons
def shell_sort(input_array):
    comparisons = 0
    comparisons += shell7(input_array)
    comparisons += shell3(input_array)
    comparisons += insertion_sort(input_array)
    return comparisons


In [30]:
test = [9,32,2,4,1,5]
insertion_sort(test)
print(test)

[1, 2, 4, 5, 9, 32]


In [31]:
test = [9,32,2,4,1,5,9,32,2,4,1,5,9,32,2,4,1,5,9,32,2,4,1,5,9,32,2,4,1,5,9,32,2,4,1,5]
print(sorted(test))
print(insertion_sort(test[:]))
print(shell_sort(test))
print(test)

[1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 9, 9, 9, 9, 9, 9, 32, 32, 32, 32, 32, 32]
320
137
[1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 9, 9, 9, 9, 9, 9, 32, 32, 32, 32, 32, 32]


4. Implement the two versions of MergeSort that we discussed in class. Create a
table or a plot for the total number of comparisons to sort the data (using data
set here) for both cases. Explain.


In [68]:
# https://www.geeksforgeeks.org/merge-sort/
def merge(input_array, left, middle, right):
    N1 = middle - left + 1 
    N2 =  right - middle 
    
    left_sub = []
    right_sub = []
    for i in range(0, N1): 
        left_sub.append(input_array[left + i]) 
    for j in range(0, N2): 
        right_sub.append(input_array[middle + 1 + j])
        
    i = j = 0
    k = left
    
    while i < N1 and j < N2:
        if left_sub[i] <= right_sub[j]:
            input_array[k] = left_sub[i]
            i += 1
        else:
            input_array[k] = right_sub[j]
            j += 1
        k += 1
    
    while i < N1:
        input_array[k] = left_sub[i]
        i += 1
        k += 1
    
    while j < N2:
        input_array[k] = right_sub[j]
        j += 1
        k += 1
    
def recursive_mergesort(input_array, left, right):
    if right > left:
        mid_point = (left + right) // 2
        recursive_mergesort(input_array, left, mid_point)
        recursive_mergesort(input_array, mid_point+1, right)
        merge(input_array, left, mid_point, right)
        
# https://www.geeksforgeeks.org/iterative-merge-sort/
def iterative_mergesort(input_array):
    sz = 1
    
    while sz < len(input_array)-1:
        left = 0
        while left < len(input_array)-1:
            mid = left + sz - 1
            
            if 2*sz + left - 1 > len(input_array)-1:
                right = len(input_array) - 1
            else:
                right = 2 * sz + left - 1
                
            merge(input_array, left, mid, right)
            left = left + sz*2
        sz = 2*sz

def mergesort_insertion_cutoff(input_array, left, right, cutoff=7):
    if right > left:
        if right - left < cutoff:
            insertion_sort(input_array)
            return
        mid_point = (left + right) // 2
        mergesort_insertion_cutoff(input_array, left, mid_point)
        mergesort_insertion_cutoff(input_array, mid_point+1, right)
        merge(input_array, left, mid_point, right)

In [69]:
test = [9,32,2,4,1,5,9,32,2,4,1,5,7,8,9,6,9,32,2,4,1,5,9,32,2,4,1,5,7,8,9,6,9,32,2,4,1,5,9,32,2,8,9,6,9,32,2,4,1,5,9,32,2,4,1,5,7,8,9,6,9,32,2,4,1,5,9,32,2,8,9,6,9,32,2,4,1,5,9,32,2,4,1,5,7,8,9,6,9,32,2,4,1,5,9,32,2,8,9,6,9,32,2,4,1,5,9,32,2,4,1,5,7,8,9,6,9,32,2,4,1,5,9,32,2,8,9,6,9,32,2,4,1,5,9,32,2,4,1,5,7,8,9,6,9,32,2,4,1,5,9,32,2,8,9,6,9,32,2,4,1,5,9,32,2,4,1,5,7,8,9,6,9,32,2,4,1,5,9,32,2,4,1,5,7,8,9,6]
print(sorted(test))
mergesort_insertion_cutoff(test, 0, len(test)-1)
print(test)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 

In [62]:
test = [9,32,2,4,1,5,9,32,2,4,1,5]
print(sorted(test))
recursive_mergesort(test, 0, len(test)-1)
print(test)

[1, 1, 2, 2, 4, 4, 5, 5, 9, 9, 32, 32]
[1, 1, 2, 2, 4, 4, 5, 5, 9, 9, 32, 32]


In [64]:
test = [9,32,2,4,1,5,9,32,2,4,1,5,7,8,9,6]
print(sorted(test))
iterative_mergesort(test)
print(test)

[1, 1, 2, 2, 4, 4, 5, 5, 6, 7, 8, 9, 9, 9, 32, 32]
[1, 1, 2, 2, 4, 4, 5, 5, 6, 7, 8, 9, 9, 9, 32, 32]


5. Implement Quicksort using median-of-three to determine the partition
element. Compare the performance of Quicksort with the Mergesort implementation
and dataset from Q4. Is there any noticeable difference when you use N=7 as the
cut-off to insertion sort. Experiment if there is any value of "cut-off to
insertion" at which the performance inverts.

In [70]:
# https://www.geeksforgeeks.org/quick-sort/
# https://stackoverflow.com/questions/50912873/python-quicksort-with-median-of-three

def median_of_three(input_array, left, right):
    middle = (left + right - 1) // 2
    a = input_array[left]
    b = input_array[middle]
    c = input_array[right-1]
    if a <= b <= c:
        return b, middle
    if c <= b <= a:
        return b, middle
    if a <= c <= b:
        return c, right-1
    if b <= c <= a:
        return c, right-1
    return a, left

def partition(input_array, left, right):
    pivot, pivot_index = median_of_three(input_array, left, right) 
    input_array[left], input_array[pivot_index] = input_array[pivot_index], input_array[left]
    i = left + 1
    for j in range(left+1, right):
        if input_array[j] < pivot:
            input_array[i], input_array[j] = input_array[j], input_array[i]
            i += 1
    input_array[left], input_array[i-1] = input_array[i-1], input_array[left]
    return i-1
    
def quicksort(input_array, left, right):
    if left < right:
        partition_index = partition(input_array, left, right)
        quicksort(input_array, left, partition_index)
        quicksort(input_array, partition_index+1, right)
        

def quicksort_insertion_cutoff(input_array, left, right, cutoff=7):
    if left < right:
        if right - left < cutoff:
            insertion_sort(input_array)
            return
        partition_index = partition(input_array, left, right)
        quicksort_insertion_cutoff(input_array, left, partition_index)
        quicksort_insertion_cutoff(input_array, partition_index+1, right)

In [35]:
test = [9,32,2,4,1,5,9,32,2,4,1,5,7,8,9,6]
print(sorted(test))
quicksort(test, 0, len(test))
print(test)

[1, 1, 2, 2, 4, 4, 5, 5, 6, 7, 8, 9, 9, 9, 32, 32]
[1, 1, 2, 2, 4, 4, 5, 5, 6, 7, 8, 9, 9, 9, 32, 32]


In [71]:
test = [9,32,2,4,1,5,9,32,2,4,1,5,7,8,9,6,9,32,2,4,1,5,9,32,2,4,1,5,7,8,9,6,9,32,2,4,1,5,9,32,2,4,1,5,7,8,9,6]
print(sorted(test))
quicksort_insertion_cutoff(test, 0, len(test))
print(test)

[1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 6, 6, 6, 7, 7, 7, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 32, 32, 32, 32, 32, 32]
[1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 6, 6, 6, 7, 7, 7, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 32, 32, 32, 32, 32, 32]


3. Create a data set of 8192 entries which has in the following order: 1024 repeats of 1, 2048 repeats of 11, 4096 repeats of 111 and 1024 repeats of 1111. Write a sort algorithm that you think will sort this set "most" effectively. Explain why you think so.

In [16]:
import csv

dataset = 1024*[1] + 2048*[11] + 4096*[111] + 1024*[1111]
# https://stackoverflow.com/questions/2084069/create-a-csv-file-with-values-from-a-python-list
with open('Q3_dataset.csv', 'w') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(dataset)

In [11]:
# Dataset is already sorted so we can use a modified bubble sort to exit after a single pass through
# The idea here is that bubble sort is completed when it makes a pass with no swaps
# https://www.quora.com/What-is-modified-bubble-sorting
def bubblesort(input_array):
    total_swaps = 0
    for i in range(len(input_array)):
        swaps = 0
        for j in range(len(input_array)-i-1):
            if input_array[j] > input_array[j+1]:
                temp = input_array[j]
                input_array[j] = input_array[j+1]
                input_array[j+1] = temp
                swaps = swaps + 1
                total_swaps = total_swaps + 1
        if swaps == 0:
            break
    return total_swaps

bubblesort(dataset)

0

2. The Kendall Tau distance is a variant of the "number of inversions" we
discussed in class. It is defined as the number of pairs that are in different
order in two permutations. Write an efficient program that computes the Kendall
Tau distance in less than quadratic time on average. Plot your results and
discuss. Use the dataset provided here. Note: data0.* for convenience is an
ordered set of numbers (in powers of two). data1.* are shuffled data sets of
sizes (as given by "*").

In [12]:
def kd_merge(input_array, left, middle, right):
    swaps = 0
    
    N1 = middle - left + 1 
    N2 = right - middle 
    
    left_sub = []
    right_sub = []
    for i in range(0, N1): 
        left_sub.append(input_array[left + i]) 
    for j in range(0, N2): 
        right_sub.append(input_array[middle + 1 + j])
        
    i = j = 0
    k = left
    
    while i < N1 and j < N2:
        if left_sub[i] <= right_sub[j]:
            input_array[k] = left_sub[i]
            i += 1
        else:
            input_array[k] = right_sub[j]
            swaps = swaps + middle - (left + i) + 1
            j += 1
        k += 1
    
    while i < N1:
        input_array[k] = left_sub[i]
        i += 1
        k += 1
    
    while j < N2:
        input_array[k] = right_sub[j]
        j += 1
        k += 1
    
    return swaps
    
def kd_mergesort(input_array, left, right):
    swaps = 0
    if right > left:
        mid_point = (left + right) // 2
        swaps = swaps + kd_mergesort(input_array, left, mid_point)
        swaps = swaps + kd_mergesort(input_array, mid_point+1, right)
        swaps = swaps + kd_merge(input_array, left, mid_point, right)
    return swaps

def kendalltau(input_array):
    temp_array = input_array[:]
    return kd_mergesort(temp_array, 0, len(temp_array)-1)

In [13]:
test = [9,32,2,4,1,5,9,32,2,4,1,5,7,8,9,6,1,2,1,2]

print(kendalltau(test))
print(bubblesort(test))

105
105


In [28]:
import csv
import random

for power in range(3, 21):
    length = 2**power
    dataset = [random.randint(0,1000) for i in range(length)]
    with open('{}_unordered_rand_int.csv'.format(length), 'w') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        wr.writerow(dataset)